In [92]:
# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
from nltk import ngrams # n-grams
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

# Timer Start 
start = time.time()

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# reading the file into Python
strt = pd.read_csv('/Users/takahiroyamada/Desktop/Challenge/search_engine/bulk_export_20220804 (1)/organization_descriptions.csv', 
                   usecols=['uuid','description'])

# read data limit to resuce the data size 
#strt =  strt[:1000]

# infomation about data frame
print(strt.info(verbose = True))

#counting null data 
print(strt.isnull().sum())

#Timer show the time 
print("Time taken: %s seconds" %(time.time() - start))

# outputting the first five rows of the dataset
strt.head(n=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038380 entries, 0 to 1038379
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   uuid         1038380 non-null  object
 1   description  1038369 non-null  object
dtypes: object(2)
memory usage: 15.8+ MB
None
uuid            0
description    11
dtype: int64
Time taken: 6.865004062652588 seconds


,uuid,description
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint is a technology platform company that...
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,"Zoho offers a suite of business, collaboration..."
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg Inc. operates a website that enables its ...
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,"Pierre Omidyar, the founder of eBay, and his w..."
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta is a social technology company that enabl...
5,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures is an early stage venture cap...
6,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel is an early and growth-stage venture cap...
7,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital provides late-stage venture c...
8,60485007-8856-bbac-aa1b-c535c41f5f47,"Currently in public beta, Omnidrive makes it e..."
9,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni is an online community of casual and expe...


In [95]:
start = time.time()
#drop the nulld data in target colmun
strt = strt.dropna(subset=['description'])
strt_rdy =  strt['description']


indices = pd.Series(strt.index, index = strt['uuid']).drop_duplicates()
vectorizer = TfidfVectorizer(stop_words='english') # set vectorizser
tfidf_matrix = vectorizer.fit_transform(strt_rdy) # fit the vectorizer to the dataframe (40 sec)
print("Time taken: %s seconds" %(time.time() - start))

Time taken: 43.416094064712524 seconds


In [96]:
a = 'e1393508-30ea-8a36-3f96-dd3226033abd'
b = 'bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7'
c =  '5f2b40b8-d1b3-d323-d81a-b7a8e89553d0'

###########################################
name = a
###########################################

start = time.time()

strt['index'] = strt.reset_index().index

name_row = strt[strt['uuid']== name]['index']
nsmr_tfidf_matrix = tfidf_matrix[name_row]
cosine_sim = linear_kernel(tfidf_matrix, nsmr_tfidf_matrix)
#cosine_sim = cosine_similarity(tfidf_matrix, nsmr_tfidf_matrix)
#using liner_kernel since it is faster than Cosine with similer result.
cosine_sim = pd.DataFrame(cosine_sim) # dataframing the Cos score


#idx = indices[name]
#sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = list(enumerate(cosine_sim[0]))
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
sim_scores = sim_scores[1:11]
startup_indices = [i[0] for i in sim_scores]
strt['uuid'].iloc[startup_indices]

print("Time taken: %s seconds" %(time.time() - start))
print(strt['uuid'].iloc[startup_indices])
# END


Time taken: 1.221313714981079 seconds
987829    1ded13c4-37de-426a-963b-e3898a2e603e
6603      b4a01f17-5256-b96c-9dcb-e13eb83129c7
915813    99e26a82-965c-4272-8dc7-2aa6f6d99b65
920078    43c309e6-106e-4eae-9ef7-690cd00f7745
4         df662812-7f97-0b43-9d3e-12f64f504fbb
645090    7dcf6e68-9fd1-435e-9da1-a5b65feb63b5
863445    db4e93e7-acca-4c01-94a4-4eba73bebdf9
867019    d4a30d4b-8884-4511-bfdf-f9d46ade8a84
834703    ac2f4df0-cf35-4bf4-a03a-57a4e2f4c3e4
743234    bf0f1624-67f3-49f7-978d-b04494100eba
Name: uuid, dtype: object


In [86]:
#UDF version 






strt['index'] = strt.reset_index().index

def get_recommend(name):
    
    name_row = strt[strt['uuid']== name]['index']
    nsmr_tfidf_matrix = tfidf_matrix[name_row]
    cosine_sim = linear_kernel(tfidf_matrix, nsmr_tfidf_matrix)
    #cosine_sim = cosine_similarity(tfidf_matrix, nsmr_tfidf_matrix)
    cosine_sim = pd.DataFrame(cosine_sim)


    #idx = indices[name]
    #sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    startup_indices = [i[0] for i in sim_scores]
    return strt['uuid'].iloc[startup_indices]

print(get_recommend(a))
print("Time taken: %s seconds" %(time.time() - start))


987829    1ded13c4-37de-426a-963b-e3898a2e603e
6603      b4a01f17-5256-b96c-9dcb-e13eb83129c7
915813    99e26a82-965c-4272-8dc7-2aa6f6d99b65
920078    43c309e6-106e-4eae-9ef7-690cd00f7745
4         df662812-7f97-0b43-9d3e-12f64f504fbb
645090    7dcf6e68-9fd1-435e-9da1-a5b65feb63b5
863445    db4e93e7-acca-4c01-94a4-4eba73bebdf9
867019    d4a30d4b-8884-4511-bfdf-f9d46ade8a84
834703    ac2f4df0-cf35-4bf4-a03a-57a4e2f4c3e4
743234    bf0f1624-67f3-49f7-978d-b04494100eba
Name: uuid, dtype: object
Time taken: 33.48547911643982 seconds


In [82]:
#Export the resupt as a CSV form
strt_r = strt[strt['uuid'].isin(get_recommend(a))].drop(df.index[[1, 3, 5]]
#strt_r = strt[strt['uuid'].isin(strt['uuid'].iloc[startup_indices])]
strt_r.to_csv('/Users/takahiroyamada/Desktop/Challenge/search_engine.csv')
strt_r


,uuid,description
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta is a social technology company that enabl...
6603,b4a01f17-5256-b96c-9dcb-e13eb83129c7,All Widgets is a company born out of a small t...
645090,7dcf6e68-9fd1-435e-9da1-a5b65feb63b5,Media & Entertainment
743234,bf0f1624-67f3-49f7-978d-b04494100eba,Entertainment Facebook Page
834703,ac2f4df0-cf35-4bf4-a03a-57a4e2f4c3e4,US-based Entertainment & Media Company
863445,db4e93e7-acca-4c01-94a4-4eba73bebdf9,A media with entertainment
867019,d4a30d4b-8884-4511-bfdf-f9d46ade8a84,Media and Entertainment
915813,99e26a82-965c-4272-8dc7-2aa6f6d99b65,it is a media/entertainment company
920078,43c309e6-106e-4eae-9ef7-690cd00f7745,News and Entertainment Media
987829,1ded13c4-37de-426a-963b-e3898a2e603e,Wetpaint is a marketing agency that provides c...
